# Predikcija plata u ekonomski razvijenim državama

**Autori:** Katarina Perović E2 131/2024, Milica Petrović E2 124/2024, Ana Radovanović E2 158/2024

**Predmet:** SIAP 

**1.Uvoz biblioteka**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys  
from IPython.display import display
!{sys.executable} -m pip install seaborn

import seaborn as sns


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

## 2. Učitavanje, osnovna analiza i vizualizacija podataka

In [ ]:
df = pd.read_csv("Salary.csv")
df.head()
df.info()
df.describe()

In [ ]:
df["Education Level"] = df["Education Level"].replace(
    ["Bachelor's Degree", "Master's Degree", "phD"],
    ["Bachelor's", "Master's", "PhD"]
)

edu_order = ["High School", "Bachelor's", "Master's", "PhD"]
df["Education Level"] = pd.Categorical(
    df["Education Level"], 
    categories=edu_order, 
    ordered=True
)

num_cols = ["Age", "Years of Experience", "Salary"]
df[num_cols].hist(bins=30, figsize=(12,6), edgecolor="black")
plt.suptitle("Distribucije numeričkih varijabli")
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(15, 4)) 

sns.countplot(
    x="Education Level", hue="Education Level", data=df, 
    order=edu_order, legend=False,
    palette="muted", ax=axes[0]
)
axes[0].set_title("Distribucija obrazovanja")
axes[0].set_xlabel("Education Level")
axes[0].set_ylabel("Broj ljudi")

sns.barplot(
    x="Education Level", y="Salary", hue="Education Level", data=df,
    estimator="mean", order=edu_order, legend=False,
    palette="muted", ax=axes[1]
)
axes[1].set_title("Prosečna plata po obrazovanju")
axes[1].set_xlabel("Education Level")
axes[1].set_ylabel("Prosečna plata")

sns.barplot(
    x="Gender", y="Salary", hue="Gender", data=df,
    estimator="mean", legend=False,
    palette="muted", ax=axes[2]
)
axes[2].set_title("Prosečna plata po polu")
axes[2].set_xlabel("Pol")
axes[2].set_ylabel("Prosečna plata")

plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
corr = df[["Age", "Years of Experience", "Salary"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korelaciona matrica")
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(14, 5)) 

sns.boxplot(
    x="Education Level", y="Salary", hue="Education Level",
    data=df, order=edu_order, legend=False,
    palette="muted", ax=axes[0]
)
axes[0].set_title("Raspon plata po obrazovanju")

sns.boxplot(
    x="Gender", y="Salary", hue="Gender",
    data=df, legend=False,
    palette="muted", ax=axes[1]
)
axes[1].set_title("Raspon plata po polu")

plt.tight_layout()
plt.show()



## 3. Pretprocesiranje podataka

In [ ]:

df = df.dropna()
df = df[~(df["Gender"] == "Other")]
df = df.drop(columns=["Race", "Country"], errors="ignore")

X = df.drop(columns=["Salary"])
y = df["Salary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

job_counts = df['Job Title'].value_counts()

total = len(df)

cumulative = job_counts.cumsum()
threshold = 0.9 * total


main_jobs = cumulative[cumulative <= threshold].index.tolist()

X_train['Job Title'] = X_train['Job Title'].where(X_train['Job Title'].isin(main_jobs), 'Other')
X_valid['Job Title'] = X_valid['Job Title'].where(X_valid['Job Title'].isin(main_jobs), 'Other')
X_test['Job Title']  = X_test['Job Title'].where(X_test['Job Title'].isin(main_jobs), 'Other')


y_train_log = np.log1p(y_train)
y_valid_log = np.log1p(y_valid)
y_test_log  = np.log1p(y_test)


gender_col = ["Gender"]
edu_col = ["Education Level"]
job_col = ["Job Title"]

edu_order = [["High School", "Bachelor's", "Master's", "PhD"]]


preprocessor = ColumnTransformer(
    transformers=[
        ('edu', OrdinalEncoder(categories=edu_order), edu_col),
        ('gender', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), gender_col),
        ('job', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), job_col)
    ],
    remainder='passthrough'  
)

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('scaler', StandardScaler())
])

X_train_scaled = pipeline.fit_transform(X_train)
X_valid_scaled = pipeline.transform(X_valid)
X_test_scaled  = pipeline.transform(X_test)

param_grid = {
    'n_neighbors': [5, 10, 15, 20, 30],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

knn = KNeighborsRegressor()
grid = GridSearchCV(knn, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid.fit(X_train_scaled, y_train_log)

print("Najbolji parametri:", grid.best_params_)

best_knn = grid.best_estimator_


y_pred_valid_log = best_knn.predict(X_valid_scaled)
y_pred_valid = np.expm1(y_pred_valid_log)  

valid_rmse = mean_squared_error(y_valid, y_pred_valid, squared=False)
valid_r2 = r2_score(y_valid, y_pred_valid)
print(f"RMSE na validacionom skupu: {valid_rmse:.2f}")
print(f"R² na validacionom skupu: {valid_r2:.4f}")


y_pred_test_log = best_knn.predict(X_test_scaled)
y_pred_test = np.expm1(y_pred_test_log)

test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
test_r2 = r2_score(y_test, y_pred_test)
print(f"RMSE na test skupu: {test_rmse:.2f}")
print(f"R² na test skupu: {test_r2:.4f}")
